# Assistente Virtual

Este notebook contém um assistente virtual local com suporte a:
- **STT (Speech to Text)**: Google Speech Recognition
- **TTS (Text to Speech)**: pyttsx3
- **Comandos**: Wikipedia, YouTube, Farmácia próxima

In [ ]:
import argparse
from typing import Protocol, Optional, Iterable
from dataclasses import dataclass
import os
import urllib.parse
import webbrowser
from dotenv import load_dotenv

load_dotenv()

In [ ]:
class SpeechToText(Protocol):
    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        pass

class TextToSpeech(Protocol):
    def speak(self, text: str) -> None:
        pass

In [ ]:
class SpeechRecognitionSTT:
    def __init__(self, language: str = "pt-BR"):
        import speech_recognition as sr
        self._sr = sr
        self._rec = sr.Recognizer()
        self._language = language

    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        print("\n[Ouvindo...] Fale agora.")
        try:
            with self._sr.Microphone() as source:
                self._rec.adjust_for_ambient_noise(source)
                audio = self._rec.listen(source, timeout=timeout)
            print("[Processando...]")
            text = self._rec.recognize_google(audio, language=self._language)
            return text
        except Exception as e:
            print(f"Erro no reconhecimento: {e}")
            return None

class TextInputSTT:
    def __init__(self, inputs: Optional[Iterable[str]] = None):
        self._inputs = list(inputs) if inputs is not None else None

    def listen(self, timeout: Optional[float] = None) -> Optional[str]:
        if self._inputs is not None:
            if not self._inputs: return None
            return self._inputs.pop(0)
        try:
            return input("\nDigite um comando (ou 'sair'): ").strip()
        except EOFError:
            return None

In [ ]:
class Pyttsx3TTS:
    def __init__(self, language: str = "pt-BR", rate: Optional[int] = None):
        import pyttsx3
        self._engine = pyttsx3.init()
        self._language = language
        if rate is not None:
            self._engine.setProperty("rate", rate)
        self._select_voice()

    def _select_voice(self) -> None:
        voices = self._engine.getProperty("voices")
        chosen = None
        for v in voices:
            name = getattr(v, "name", "") or ""
            lang = "".join(getattr(v, "languages", []) or [])
            if self._language.lower()[:2] in (lang.lower(), name.lower()):
                chosen = v.id
                break
        if chosen:
            self._engine.setProperty("voice", chosen)

    def speak(self, text: str) -> None:
        print(f"🤖 Assistente: {text}")
        try:
            self._engine.say(text)
            self._engine.runAndWait()
        except Exception as e:
            print(f"(Erro áudio: {e})")

class SilentTTS:
    def speak(self, text: str) -> None:
        print(f"🤖 Assistente (silencioso): {text}")

In [ ]:
@dataclass
class ActionResult:
    success: bool
    message: str

def parse_and_execute(text: str) -> ActionResult:
    s = (text or "").lower()
    if not s: return ActionResult(False, "Nenhum texto reconhecido")
    
    if "wikipedia" in s:
        q = s.replace("wikipedia", "").replace("pesquisar", "").strip()
        url = "https://pt.wikipedia.org/wiki/Special:Search?search=" + urllib.parse.quote_plus(q)
        webbrowser.open(url)
        return ActionResult(True, f"Pesquisando Wikipedia: {q}")
        
    if "youtube" in s or "video" in s:
        q = s.replace("youtube", "").replace("video", "").replace("pesquisar", "").strip()
        url = "https://www.youtube.com/results?search_query=" + urllib.parse.quote_plus(q)
        webbrowser.open(url)
        return ActionResult(True, f"Pesquisando YouTube: {q}")
        
    if "farmácia" in s or "farmacia" in s:
        webbrowser.open("https://www.google.com/maps/search/farmacia+perto+de+mim")
        return ActionResult(True, "Abrindo farmácias próximas")
        
    return ActionResult(False, "Comando não reconhecido")

In [ ]:
class Assistant:
    def __init__(self, stt: SpeechToText, tts: TextToSpeech):
        self._stt = stt
        self._tts = tts

    def run(self):
        self._tts.speak("Olá! Como posso ajudar?")
        while True:
            text = self._stt.listen()
            if not text: continue
            print(f"🎤 Você: {text}")
            
            if text.lower().strip() in ["sair", "encerrar", "tchau"]:
                self._tts.speak("Até logo!")
                break
                
            result = parse_and_execute(text)
            self._tts.speak(result.message)

In [ ]:
# Para rodar no notebook, usamos entrada de texto por padrão
stt = TextInputSTT()
try:
    tts = Pyttsx3TTS()
except:
    tts = SilentTTS()

assistant = Assistant(stt, tts)
assistant.run()